In [1]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
import pickle as pk

In [2]:
mnist_data = fetch_openml('mnist_784')
X, y = mnist_data["data"], mnist_data["target"]

In [ ]:
# plt.imshow(X.to_numpy()[170, :].reshape(28, 28))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int), test_size=0.2, random_state=42)

#### Árvore de decisão

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
param_grid_tree = {
    'max_depth': [5, 10],
    'min_samples_split': [2, 10, 20],
    'min_samples_leaf': [1, 5, 10]
}

grid_search_tree = GridSearchCV(tree_clf, param_grid_tree, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search_tree.fit(X_train, y_train)

print("Best parameters:", grid_search_tree.best_params_)

In [24]:
grid_search_tree.best_params_

{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [15]:
y_pred_tree = grid_search_tree.predict(X_test)
acc_tree = accuracy_score(y_test, y_pred_tree)
acc_tree

0.8582857142857143

#### Random forest

In [4]:
from sklearn.ensemble import RandomForestClassifier

In [5]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5, min_samples_leaf=2, random_state=42)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [ ]:
#################### Máquina não possui memória suficiente para realizar um Gridsearch e avaliar os melhores parâmetros ####################

# rf_clf = RandomForestClassifier(random_state=42)
# param_grid_rf = {
#     'n_estimators': [100, 200, 300],
#     'max_depth': [5, 10, 20],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }

# rand_search_rf = RandomizedSearchCV(rf_clf, param_distributions=param_grid_rf, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
# rand_search_rf.fit(X_train, y_train)

# print("Best parameters:", rand_search_rf.best_params_)

In [6]:
y_pred_rf = rf_clf.predict(X_test)

acc_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest accuracy: {acc_rf:.2f}")

Random Forest accuracy: 0.94


#### XGBoost

In [8]:
import xgboost as xgb

In [ ]:
xgb_clf = xgb.XGBClassifier(n_estimators=100, max_depth=10, learning_rate=1, subsample=1, random_state=42, verbosity=1, n_jobs=-1)
xgb_clf.fit(X_train, y_train)

In [ ]:
#################### Máquina não possui memória suficiente para realizar um Gridsearch e avaliar os melhores parâmetros ####################

# xgb_clf = xgb.XGBClassifier(random_state=42)

# param_grid_xgb = {
#     'n_estimators': [50, 100, 125],
#     'max_depth': [3, 6, 9],
#     'learning_rate': [0.1, 0.5, 1],
#     'subsample': [0.6, 1.0],
#     'colsample_bytree': [0.6, 1.0],
# }

# rand_search_xgb = RandomizedSearchCV(estimator=xgb_clf, param_distributions=param_grid_xgb, cv=3, n_jobs=-1, verbose=2)
# rand_search_xgb.fit(X_train, y_train)

# print("Best parameters:", rand_search_xgb.best_params_)

In [ ]:
y_pred_xgb = xgb_clf.predict(X_test)

acc_xgb = accuracy_score(y_test, y_pred_xgb)
print(f"Random Forest accuracy: {acc_xgb:.2f}")

Random Forest accuracy: 0.97


In [13]:
# with open('../modelos/model_xgb.pkl', 'wb') as pkl:
#     pk.dump(xgb_clf, pkl)

#### Visualização da Árvore de Decisão e Medida de Impureza

In [93]:
#arvore de decisao treinada acima com 10 niveis de profundidade tem dificil visualização, portanto, para a análise desta seção
model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1, min_samples_split=2, criterion='gini', min_impurity_decrease=0.001) #0.01, 0.05, 0.001, 0.1
model.fit(X_train, y_train)

model_pred = model.predict(X_test)

In [94]:
accuracy_score(y_test, model_pred)

0.6577142857142857

O parâmetro min_impurity_decrease define o valor mínimo na variação do coeficiente de impureza do nó durante a avaliação da feature que deve ser considerado para que seja feito um split. O aumento desse valor implica em uma maior 
tolerância à variações de impureza e portanto em uma árvore com menor profundidade, o que reduz o overfit do modelo aos dados e aumenta a capacidade de generalização. Valores muito altos levam a uma árvore muito reduzida e sem precisão.

In [96]:
#### Analisar como a medida de impureza afeta a escolha dos atributos para dividir os dados em cada nó da árvore
tree = model.tree_
feature_indices = tree.feature
thresholds = tree.threshold

for i in range(tree.node_count):
    if tree.children_left[i] != tree.children_right[i]:
        print(f"Nó {i}: Split da feature {feature_indices[i]} em {thresholds[i]}")

Nó 0: Split da feature 350 em 126.5
Nó 1: Split da feature 435 em 0.5
Nó 2: Split da feature 597 em 1.5
Nó 3: Split da feature 486 em 0.5
Nó 4: Split da feature 404 em 0.5
Nó 7: Split da feature 571 em 43.5
Nó 10: Split da feature 489 em 16.5
Nó 11: Split da feature 380 em 2.5
Nó 14: Split da feature 347 em 2.5
Nó 17: Split da feature 542 em 1.5
Nó 18: Split da feature 432 em 0.5
Nó 19: Split da feature 486 em 20.5
Nó 22: Split da feature 211 em 7.5
Nó 25: Split da feature 270 em 0.5
Nó 26: Split da feature 273 em 1.0
Nó 29: Split da feature 347 em 0.5
Nó 32: Split da feature 489 em 26.5
Nó 33: Split da feature 290 em 34.5
Nó 34: Split da feature 486 em 43.5
Nó 35: Split da feature 296 em 2.5
Nó 38: Split da feature 657 em 4.5
Nó 41: Split da feature 297 em 2.5
Nó 42: Split da feature 486 em 58.0
Nó 45: Split da feature 598 em 0.5
Nó 48: Split da feature 347 em 1.5
Nó 49: Split da feature 206 em 0.5
Nó 50: Split da feature 457 em 1.5
Nó 53: Split da feature 542 em 88.5
Nó 56: Split da 

In [97]:
#visualização da árvore
from sklearn.tree import plot_tree

plt.figure(figsize=(200,100))
plot_tree(model, filled=True)
plt.show() #salvo em arvore_decisao.png

In [34]:
#Calcular a medida de impureza para diferentes conjuntos de dados e entender como ela impacta o desempenho do modelo.
# def gini_impurity(labels):
#     proportions = np.bincount(labels) / len(labels)
#     return 1 - np.sum(proportions**2)